# Ensemble Evaluation Script

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from imutils import paths
import glob
import os

In [3]:
# construct the argument parser
data_dir = r"C:\Users\mhasa\Google Drive\Tutorial Corner\PYTH\DeepLearning\DeepLearning-DL4CV\ImageDatasets\biblio2chap6\models"

args_dict = {
    "model": f"{data_dir}"
}


In [4]:
# load just the test data

testx, testy = cifar10.load_data()[1]
testx = testx.astype("float32") / 255.0

class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# binarise the labels
le = LabelBinarizer()
testy = le.fit_transform(testy)

In [5]:
# collect all model files
model_paths = os.path.sep.join([args_dict["model"], "*.pt"])
model_paths = list(glob.glob(model_paths))

In [6]:
# loop over paths and load models
models = []

for i, model_path in enumerate(model_paths):
    print(f"[INFO] loading model {i+1}/{len(model_paths)}")
    models.append(load_model(model_path))



[INFO] loading model 1/1


OSError: SavedModel file does not exist at: C:\Users\mhasa\Google Drive\Tutorial Corner\PYTH\DeepLearning\DeepLearning-DL4CV\ImageDatasets\biblio2chap6\models\sdd.pt/{saved_model.pbtxt|saved_model.pb}

In [ ]:
# now evaluate the ensemble
print(f"[INFO] evaluate the ensemble.....")
predictions = []

# lood over the models
for model in models:
    predictions.append(model.predict(testx, batch_size=64))

preds = np.average(predictions, axis=0)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=class_names))